In [1]:
import numpy as np
import pandas as pd
from sklearn import neighbors, datasets
from sklearn import model_selection
from sklearn import metrics
from sklearn import preprocessing
from sklearn.decomposition import PCA
!pip install seaborn
!pip install yellowbrick
import numpy as np
import pandas as pd
import datetime
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt, numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import AgglomerativeClustering
from matplotlib.colors import ListedColormap
from sklearn import metrics
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
np.random.seed(42)


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
df = pd.read_csv('marketing_campaign.csv', sep='\t')

In [3]:
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


# Code added by Taufiq 

In [4]:
#income replace with median:
df['Income'].fillna(df['Income'].median(), inplace=True)

In [5]:
# 3. Total Number of Purchases a single customer make
df['Total_Purchases'] = df[['NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumDealsPurchases']].sum(axis=1)

In [6]:
# Applying one-hot encoding to the 'Education' categorical variable
from sklearn.preprocessing import OneHotEncoder
education_ohe = pd.get_dummies(df['Education'], prefix='Education')

# Adding the one-hot encoded columns to the main DataFrame
df = pd.concat([df, education_ohe], axis=1)

# Displaying the first few rows to verify the new columns
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,Complain,Z_CostContact,Z_Revenue,Response,Total_Purchases,Education_2n Cycle,Education_Basic,Education_Graduation,Education_Master,Education_PhD
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,0,3,11,1,25,False,False,True,False,False
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,0,3,11,0,6,False,False,True,False,False
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,0,3,11,0,21,False,False,True,False,False
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,0,3,11,0,8,False,False,True,False,False
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,0,3,11,0,19,False,False,False,False,True


In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
import matplotlib.pyplot as plt
df.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
#corr_matrix =df.corr()
# Select only numeric columns
numeric_df = df.select_dtypes(include=['float64', 'int64'])

# Compute correlation matrix
corr_matrix = numeric_df.corr()

In [ ]:
print(corr_matrix)

In [ ]:
df["Year_Birth"].value_counts()

In [ ]:
df["Education"].value_counts()

In [ ]:
df["Marital_Status"].value_counts()

In [ ]:
df["Kidhome"].value_counts()

In [ ]:
df["Teenhome"].value_counts()

In [ ]:
print(df.isnull().sum())

In [ ]:
df=df.dropna(subset=["Income"]) 

In [ ]:
print(df.isnull().sum())

In [ ]:
from pandas.plotting import scatter_matrix
attributes = ["Income", "MntWines", "MntFruits",
 "MntMeatProducts","MntFishProducts","MntSweetProducts","MntGoldProds"]
scatter_matrix(df[attributes], figsize=(12, 8))

In [ ]:
df["Dt_Customer"] = pd.to_datetime(df["Dt_Customer"], format='mixed')
dates = []
for i in df["Dt_Customer"]:
    i = i.date()
    dates.append(i)  
#Dates of the newest and oldest recorded customer
print("The newest customer's enrolment date in therecords:",max(dates))
print("The oldest customer's enrolment date in the records:",min(dates))

In [ ]:
#creation d'une feature "Customer_For"
days = []
d1 = max(dates) #taking it to be the newest customer
for i in dates:
    delta = d1 - i
    days.append(delta)
df["Customer_For"] = days
df["Customer_For"] = pd.to_numeric(df["Customer_For"], errors="coerce")

In [ ]:
df.describe()

In [ ]:
df["Age"] = 2020-df["Year_Birth"]

In [ ]:
df["Spent"] = df["MntWines"]+ df["MntFruits"]+ df["MntMeatProducts"]+ df["MntFishProducts"]+ df["MntSweetProducts"]+ df["MntGoldProds"]

#Deriving living situation by marital status"Alone"
df["Living_With"]=df["Marital_Status"].replace({"Married":"Partner", "Together":"Partner", "Absurd":"Alone", "Widow":"Alone", "YOLO":"Alone", "Divorced":"Alone", "Single":"Alone",})
df["Children"]=df["Kidhome"]+df["Teenhome"]

In [ ]:
#Add by Taufiq: feature = Total Number of Purchases a single customer make
df['Total_Purchases'] = df[['NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumDealsPurchases']].sum(axis=1)

In [ ]:
df["Family_Size"] = df["Living_With"].replace({"Alone": 1, "Partner":2})+ df["Children"]

In [ ]:
#Dropping the outliers by setting a cap on Age and income. 
df = df[(df["Age"]<90)]
df = df[(df["Income"]<600000)]

In [ ]:
df["Is_Parent"] = np.where(df.Children> 0, 1, 0)
df["IncomePer"] =df["Income"]/df["Family_Size"]

In [ ]:
to_drop = ["Marital_Status", "Dt_Customer","Year_Birth","Income",'Z_CostContact','Z_Revenue']
df= df.drop(to_drop, axis=1)

In [ ]:
df["Age"].value_counts()

In [ ]:
df["Total_Promos"] = df["AcceptedCmp1"]+ df["AcceptedCmp2"]+ df["AcceptedCmp3"]+ df["AcceptedCmp4"]+ df["AcceptedCmp5"]

In [ ]:
#Get list of categorical variables
s = (df.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables in the dataset:", object_cols)

In [ ]:
#Label Encoding the object dtypes.
LE=LabelEncoder()
for i in object_cols:
    df[i]=df[[i]].apply(LE.fit_transform)
    
print("All features are now numerical")

In [ ]:
selected_attributes = ['Education', 'IncomePer', 'Kidhome','Teenhome','Age','Spent','Living_With','Children','Family_Size','Is_Parent','MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts',
                     'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases',
                     'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases',
                     'NumWebVisitsMonth']
data1 = df[selected_attributes].copy()

In [ ]:
#Scaling
scaler = StandardScaler()
scaler.fit(data1)
data1 = pd.DataFrame(scaler.transform(data1),columns= data1.columns )

print("All features are now scaled")

In [ ]:
data1.head()

In [ ]:
from sklearn.cluster import KMeans

k_values = range(1, 11) 

# Initialize an empty list to store the within-cluster sum of squares (WCSS) for each value of k
wcss = []

# Iterate over each value of k
for k in k_values:
    # Initialize the k-means clustering algorithm with the current value of k
    kmeans = KMeans(n_clusters=k, random_state=42)
    
    # Fit the k-means model to the subset data
    kmeans.fit(data1)
    
    # Append the WCSS to the list
    wcss.append(kmeans.inertia_)

# Plot the elbow curve
plt.plot(k_values, wcss, marker='o', linestyle='-', color='b')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Within-Cluster Sum of Squares (WCSS)')
plt.title('Elbow Method for Optimal k')
plt.xticks(k_values)
plt.show()

In [ ]:
#Initiating the Agglomerative Clustering model 
AC = AgglomerativeClustering(n_clusters=3)
# fit model and predict clusters
yhat_AC = AC.fit_predict(data1)
data1["Clusters"] = yhat_AC
#Adding the Clusters feature to the orignal dataframe.
df["Clusters"]= yhat_AC

In [ ]:
#Plotting countplot of clusters
pal = ["#682F2F","#B9C0C9", "#9F8A78","#F3AB60"]
pl = sns.countplot(x=df["Clusters"], palette= pal)
pl.set_title("Distribution Of The Clusters")
plt.show()

In [ ]:
pl = sns.scatterplot(data = df,x=df["Spent"], y=df["IncomePer"],hue=df["Clusters"], palette= pal)
pl.set_title("Cluster's Profile Based On Income And Spending")
plt.legend()
plt.show()

In [ ]:
#Plotting count of total campaign accepted.
plt.figure()
pl = sns.countplot(x=df["Total_Promos"],hue=df["Clusters"], palette= pal)
pl.set_title("Count Of Promotion Accepted")
pl.set_xlabel("Number Of Total Accepted Promotions")
plt.show()

In [ ]:
plt.figure()
pl = sns.countplot(x=df["Children"],hue=df["Clusters"], palette= pal)
pl.set_title("Number of Children")
pl.set_xlabel("Number of Children")
plt.show()

In [ ]:
plt.figure()
pl = sns.countplot(x=df["Is_Parent"],hue=df["Clusters"], palette= pal)
pl.set_title("is parent")
pl.set_xlabel("is parent")
plt.show()

In [ ]:
plt.figure()
pl = sns.countplot(x=df["Kidhome"],hue=df["Clusters"], palette= pal)
pl.set_title("number of kids")
pl.set_xlabel("number of kids")
plt.show()

In [ ]:
plt.figure()
pl = sns.countplot(x=df["Teenhome"],hue=df["Clusters"], palette= pal)
pl.set_title("number of teens")
pl.set_xlabel("number of teens")
plt.show()

In [ ]:
#!pip install mlxtend

In [ ]:
#data1.describe()
df.describe()

In [ ]:
percentile_25 = df['MntSweetProducts'].quantile(0.25)
#print(percentile_75)
#median_value = df['MntGoldProds'].median()
#mean_value = df['MntGoldProds'].mean()
print(percentile_25)

In [ ]:
cluster_means = df.groupby("Clusters").mean()

# Print cluster means
print("Cluster means:")
print(cluster_means)

In [ ]:
!pip install pyfpgrowth

In [ ]:
import pyfpgrowth

# Define the unique cluster numbers
clusters = df['Clusters'].unique()

# Specify the minimum support threshold
min_support = 0.1
top_supported_itemsets = {}
# Iterate over each cluster
for cluster in clusters:
    # Filter the main dataset for the current cluster
    cluster_data = df[df['Clusters'] == cluster]
    # Convert the cluster data into transaction format
    transactions = []
    for index, row in cluster_data.iterrows():
        # Create a list to store items in the transaction
        transaction = []
        if row['MntWines'] > 175.50:
            transaction.append('wine')
        if row['MntFruits'] > 8:
            transaction.append('fruits')
        if row['MntMeatProducts'] > 68:
            transaction.append('meat')
        if row['MntFishProducts'] > 12:
            transaction.append('fish')
        if row['MntSweetProducts'] >8:
            transaction.append('sweet')
        if row['MntGoldProds'] > 24.5:
            transaction.append('gold')

    transactions.append(transaction)
    
    # Apply FP-Growth algorithm to find frequent itemsets for the current cluster
    patterns = pyfpgrowth.find_frequent_patterns(transactions, min_support)
    sorted_patterns = sorted(patterns.items(), key=lambda x: x[1], reverse=True)[:3]
    top_supported_itemsets[cluster] = sorted_patterns
    # Print frequent itemsets with their support for the current cluster
    print(f"Frequent itemsets for Cluster {cluster}:")
    for itemset, support in patterns.items():
        print(f"Itemset: {itemset}, Support: {support}")
        
    print()


In [ ]:
for cluster, itemsets in top_supported_itemsets.items():
    print(f"Cluster {cluster}: Top three itemsets with the most support:")
    for itemset, support in itemsets:
        print(f"Itemset: {itemset}, Support: {support}")
    print()

In [ ]:
min_support = 0.1
top_supported_itemsets = {}
# Iterate over each cluster
for cluster in clusters:
    # Filter the main dataset for the current cluster
    cluster_data = df[df['Clusters'] == cluster]
    # Convert the cluster data into transaction format
    transactions = []
    for index, row in cluster_data.iterrows():
        # Create a list to store items in the transaction
        transaction = []
        if row['MntWines'] > 24:
            transaction.append('wine')
        if row['MntFruits'] > 2:
            transaction.append('fruits')
        if row['MntMeatProducts'] > 16:
            transaction.append('meat')
        if row['MntFishProducts'] > 3:
            transaction.append('fish')
        if row['MntSweetProducts'] > 1:
            transaction.append('sweet')
        if row['MntGoldProds'] > 1:
            transaction.append('gold')

    transactions.append(transaction)
    
    # Apply FP-Growth algorithm to find frequent itemsets for the current cluster
    patterns = pyfpgrowth.find_frequent_patterns(transactions, min_support)
    sorted_patterns = sorted(patterns.items(), key=lambda x: x[1], reverse=True)[:3]
    top_supported_itemsets[cluster] = sorted_patterns
    # Print frequent itemsets with their support for the current cluster
    print(f"Frequent itemsets for Cluster {cluster}:")
    for itemset, support in patterns.items():
        print(f"Itemset: {itemset}, Support: {support}")
        
    print()


In [ ]:
for cluster, itemsets in top_supported_itemsets.items():
    print(f"Cluster {cluster}: Top three itemsets with the most support:")
    for itemset, support in itemsets:
        print(f"Itemset: {itemset}, Support: {support}")
    print()